# Model: Pretrained Inception Network

## Preamble

In [14]:
from tensorflow import keras
import PIL
import numpy
import pandas
import itertools

## Data Preparation

In [2]:
movies = catalog.load("movies")

2021-03-15 18:00:16,179 - kedro.io.data_catalog - INFO - Loading data from `movies` (ParquetDataSet)...


**exclude test set**

In [3]:
movies_test = catalog.load("movies_test")

2021-03-15 18:00:16,464 - kedro.io.data_catalog - INFO - Loading data from `movies_test` (CSVDataSet)...


In [4]:
movies_test

,original_title,poster_path,genre_names,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,,,
17335,Obsessed,/g7kv7RXzqCSMmiYmV0gVLpxB0YH.jpg,['Drama' 'Thriller'],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
757,Murder Most Foul,/qTCKB5rxYzgdWUUH4UMgpRVH3GJ.jpg,['Comedy' 'Crime' 'Drama' 'Mystery' 'Thriller'],0,0,0,1,1,0,1,...,0,0,0,1,0,0,0,1,0,0
11886,Robin Hood,/x9AvkYek0bGdxQSZ8W3lAjGrREm.jpg,['Animation' 'Family'],0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2115,Light of Day,/hFXnASDyZd3i9TOuBPUtJRvfZsK.jpg,['Music' 'Drama'],0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1915,"Manta, Manta",/cpc3fbClSG46azA1dNW0ilKvIgj.jpg,['Comedy' 'Action'],1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,All Day Every Day,NaN,[],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18002,"Love, Honour and Obey",/n4mq89w2Dgn2oDnYGtx5qnSmfTA.jpg,['Action' 'Comedy' 'Thriller' 'Crime'],1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1830,Lord of War,/xEdwdFZRwrNAvTDx0fAV3MAInaA.jpg,['Crime' 'Drama' 'Thriller'],0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
movies_train = movies.loc[movies.index.difference(movies_test.index)]

In [6]:
movies_train

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,genre_names
id,,,,,,,,,,,,,,,,,,,,,
2,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,,Ariel,False,6.8,128,NaN,None,None,None,"[Drama, Crime, Comedy]"
3,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,,Shadows in Paradise,False,7.2,125,NaN,None,None,None,"[Drama, Comedy]"
5,False,/u0zMKKpEdDWpOKmFW2sLbKKICJH.jpg,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.7,1901,NaN,None,None,None,"[Crime, Comedy]"
6,False,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,NaN,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",...,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.5,184,NaN,None,None,None,"[Action, Thriller, Crime]"
8,False,None,NaN,42000,"[{'id': 99, 'name': 'Documentary'}]",http://www.lifeinloops.com,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,...,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.3,13,NaN,None,None,None,[Documentary]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,False,/gpEJ2fR3Oy7hAMl8wAtv6Nd40x3.jpg,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",,tt1131734,en,Jennifer's Body,"Jennifer, a gorgeous, seductive cheerleader wh...",...,She's evil... and not just high school evil.,Jennifer's Body,False,5.7,2451,NaN,None,None,None,"[Comedy, Horror]"
19996,False,None,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",,tt0932661,en,Brother's War,The story is set in the latter days of World W...,...,,Brother's War,False,3.9,14,NaN,None,None,None,"[Action, Drama]"
19997,False,/TNfPn4Fwi0Fqx4BsjxOL3Tmscw.jpg,NaN,1250000,"[{'id': 53, 'name': 'Thriller'}]",,tt0046126,en,Niagara,"Rose Loomis and her older, gloomier husband, G...",...,A raging torrent of emotion that even nature c...,Niagara,False,6.8,214,NaN,None,None,None,[Thriller]


## Image Classifier

In [7]:
class ImageFileClassifier:
    
    def __init__(self, img_dir, min_probability=0.5):
        self.model = keras.applications.inception_v3.InceptionV3()
        self.input_dim = (299,299)
        self.min_probability = min_probability
        self.img_dir = img_dir
        
    def classify(self, img_path, as_list=True):
        """
        Classify an image.
        
        img_path: path pointing to an image file
        """
        if img_path is None:
            return []
        img_path = f"{self.img_dir}{img_path}"
        try:
            with PIL.Image.open(img_path) as img:
                img = img.resize(self.input_dim)
                img_a = keras.preprocessing.image.img_to_array(img)
                img_a = keras.applications.inception_v3.preprocess_input(img_a)
                predictions = self.model.predict(
                    numpy.array([img_a])
                )
                predictions_decoded = keras.applications.imagenet_utils.decode_predictions(predictions)[0]
                predictions = pandas.DataFrame(
                    predictions_decoded,
                    columns=["class_label", "class_name", "class_probability"]
                )
                # result
                if self.min_probability:
                    predictions = predictions[predictions["class_probability"] >= self.min_probability]
            return list(predictions["class_name"])
        except Exception as ex:
            #print(ex)
            return []


In [8]:
classifier = ImageFileClassifier(img_dir="../data/01_raw/posters")

In [9]:
classifier.classify("/1Crro6hN91lXc1sWXkU52tfjpAs.jpg")

['revolver']

In [10]:
from tqdm.auto import tqdm
tqdm.pandas()

In [11]:
%%time
movies["objects"] = movies["poster_path"].apply(classifier.classify)

2021-03-15 18:00:40,652 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (None, 299, 299, 1).
2021-03-15 18:00:44,937 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (None, 299, 299, 1).
2021-03-15 18:01:04,872 - tensorflow - WARNING - Model was constructed with shape (None, 299, 299, 3) for input KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shap

In [16]:
label_counts = pandas.Series(
    itertools.chain.from_iterable(
        movies[~movies["objects"].isna()]["objects"]
    )
).value_counts()

In [28]:
label_counts[:60]

comic_book          1558
book_jacket         1168
web_site             364
bow_tie               84
suit                  67
stage                 55
sunglasses            40
wig                   37
jersey                31
military_uniform      29
cowboy_hat            29
gown                  24
bikini                24
candle                24
jean                  23
mask                  23
bobsled               20
groom                 19
sunglass              18
miniskirt             16
car_mirror            15
swimming_trunks       15
theater_curtain       15
television            15
rifle                 14
revolver              14
lab_coat              13
mortarboard           12
microphone            12
lipstick              12
brassiere             12
umbrella              12
neck_brace            12
feather_boa           12
spotlight             11
bathing_cap           11
basketball            10
toyshop               10
racer                 10
sombrero               9


In [30]:
label_counts[60:120]

volcano               6
binoculars            6
bearskin              6
fountain              6
academic_gown         6
menu                  6
great_white_shark     6
chainlink_fence       6
ballplayer            6
crash_helmet          5
sweatshirt            5
paddle                5
prison                5
pick                  5
syringe               5
cellular_telephone    5
scoreboard            5
slot                  5
dumbbell              5
parachute             5
harmonica             5
ski                   5
kimono                4
electric_guitar       4
Chihuahua             4
balloon               4
scuba_diver           4
trench_coat           4
pajama                4
hoopskirt             4
drumstick             4
sleeping_bag          4
king_penguin          4
power_drill           4
bulletproof_vest      4
snowmobile            4
analog_clock          4
digital_clock         4
assault_rifle         4
shower_cap            4
stethoscope           4
abaya           

In [32]:
#movies.to_parquet("../data/04_feature/movies_objects.parquet")

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer

In [18]:
mlb = MultiLabelBinarizer()

In [19]:
mlb.fit(movies["objects"])

MultiLabelBinarizer()

In [20]:
mlb.classes_

array(['African_crocodile', 'American_alligator', 'Appenzeller',
       'Arabian_camel', 'Band_Aid', 'Brittany_spaniel', 'Chihuahua',
       'Christmas_stocking', 'Crock_Pot', 'French_loaf',
       'German_shepherd', 'Granny_Smith', 'Indian_cobra',
       'Indian_elephant', 'Labrador_retriever', 'Model_T', 'Newfoundland',
       'Pekinese', 'Persian_cat', 'Polaroid_camera', 'Saint_Bernard',
       'Tibetan_mastiff', 'Windsor_tie', 'abacus', 'abaya',
       'academic_gown', 'accordion', 'acoustic_guitar', 'airliner',
       'airship', 'alp', 'altar', 'ambulance', 'analog_clock',
       'armadillo', 'ashcan', 'assault_rifle', 'axolotl', 'backpack',
       'balance_beam', 'balloon', 'ballplayer', 'bannister', 'barbell',
       'barbershop', 'barn', 'barrel', 'barrow', 'baseball', 'basketball',
       'basset', 'bassoon', 'bath_towel', 'bathing_cap', 'bathtub',
       'bearskin', 'beer_glass', 'bicycle-built-for-two', 'bikini',
       'binoculars', 'black_grouse', 'bloodhound', 'boathouse'

In [46]:
objects = pandas.DataFrame(
    mlb.transform(movies["objects"]),
    columns=mlb.classes_,
    index=movies.index
)
objects

,African_crocodile,American_alligator,Appenzeller,Arabian_camel,Band_Aid,Brittany_spaniel,Chihuahua,Christmas_stocking,Crock_Pot,French_loaf,...,water_jug,water_snake,water_tower,web_site,whiskey_jug,wig,window_screen,wing,wool,wreck
id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
X_train = objects.loc[movies.index.difference(movies_test.index)]

In [48]:
X_test = objects.loc[movies_test.index]

In [49]:
X_train

,African_crocodile,American_alligator,Appenzeller,Arabian_camel,Band_Aid,Brittany_spaniel,Chihuahua,Christmas_stocking,Crock_Pot,French_loaf,...,water_jug,water_snake,water_tower,web_site,whiskey_jug,wig,window_screen,wing,wool,wreck
id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X_test

,African_crocodile,American_alligator,Appenzeller,Arabian_camel,Band_Aid,Brittany_spaniel,Chihuahua,Christmas_stocking,Crock_Pot,French_loaf,...,water_jug,water_snake,water_tower,web_site,whiskey_jug,wig,window_screen,wing,wool,wreck
id,,,,,,,,,,,,,,,,,,,,,
17335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
757,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
mlb_genres = MultiLabelBinarizer()

In [60]:
mlb_genres.fit(movies["genre_names"])

MultiLabelBinarizer()

In [62]:
genres = pandas.DataFrame(
    mlb_genres.transform(movies["genre_names"]),
    columns=mlb_genres.classes_,
    index=movies.index
)


In [63]:
genres

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
19996,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [65]:
Y_train = genres.loc[movies.index.difference(movies_test.index)]

In [66]:
Y_test = genres.loc[movies_test.index]

## Network Architecture

In [71]:
n_objects = objects.shape[1]
n_genres = genres.shape[1]

In [72]:
output_layer = keras.layers.Dense(
    units=n_genres,
    activation="sigmoid"
)

In [100]:
net = keras.models.Sequential(
    [
        keras.layers.Input(
            shape=n_objects
        ),
        keras.layers.Dense(
            units=128
        ),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(
            units=n_genres,
            activation="sigmoid"
        )
    ]
)

In [101]:
net.compile(
    loss="binary_crossentropy",
    optimizer="adam",
)

In [102]:
net.fit(
    X_train,
    Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[
        keras.callbacks.EarlyStopping(
            min_delta=0.00001,
            patience=8
        )
    ]
)

Epoch 1/100
347/347 [==============================] - 1s 1ms/step - loss: 0.4787 - val_loss: 0.3126
Epoch 2/100
347/347 [==============================] - 0s 925us/step - loss: 0.3103 - val_loss: 0.3111
Epoch 3/100
347/347 [==============================] - 0s 930us/step - loss: 0.3102 - val_loss: 0.3108
Epoch 4/100
347/347 [==============================] - 0s 960us/step - loss: 0.3061 - val_loss: 0.3105
Epoch 5/100
347/347 [==============================] - 0s 961us/step - loss: 0.3063 - val_loss: 0.3104
Epoch 6/100
347/347 [==============================] - 0s 979us/step - loss: 0.3046 - val_loss: 0.3101
Epoch 7/100
347/347 [==============================] - 0s 1ms/step - loss: 0.3045 - val_loss: 0.3100
Epoch 8/100
347/347 [==============================] - 0s 1ms/step - loss: 0.3052 - val_loss: 0.3101
Epoch 9/100
347/347 [==============================] - 0s 1ms/step - loss: 0.3022 - val_loss: 0.3100
Epoch 10/100
347/347 [==============================] - 0s 1ms/step - loss: 0.303

In [115]:
Y_pred = pandas.DataFrame(
    net.predict(X_test).round(),
    columns=genres.columns,
    index=X_test.index
)


In [116]:
from sklearn.metrics import f1_score

In [117]:
f1_score(
    Y_test,
    Y_pred,
    average="micro"
)

0.10738255033557048

In [118]:
f1_score(
    Y_test,
    Y_pred,
    average="weighted"
)

0.09254567959199254

In [119]:
Y_pred

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
17335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1830,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
Y_test

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,,
17335,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
757,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0
11886,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2115,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1915,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18002,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1830,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
